In [3]:
import os
import sys
import glob
import scipy
import sklearn
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from collections import defaultdict
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

%matplotlib inline
sns.set_style('whitegrid')
pd.set_option('display.max_rows', 100)
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['pdf.fonttype'] = 42
pd.set_option('display.max_columns', 100)

sns.set_palette("Set2")


In [4]:
data_loc = '/home/mattolm/user_data/IndianMicrobiome/tables/'
Pdb = pd.read_csv(data_loc + 'IndianMicrobiome_basicinfo_v3.csv')
Pdb = Pdb.drop_duplicates()
Pdb


sample                                              read1  \
0      AK_SR_1  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
1      AK_SR_2  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
3    AK_SR_2.2  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
5      AK_SR_3  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
6      AK_SR_4  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
..         ...                                                ...   
109      TS8_B  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
110     TS12_A  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
111     TS12_B  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
112     TS14_A  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
113     TS14_B  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   

                                                 read2            library_key  \
0    s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...    LibPlate-A2_AK-SR-1   
1    s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...    LibPlate-A4_AK-SR-2   
3    s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...  LibPlate-D2_AK-SR-2-2   
5    s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...    LibPlate-A6_AK-SR-3   
6    s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...    LibPlate-A8_AK-SR-4   
..                                                 ...                    ...   
109  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...      LibPlate-P4_TS8-B   
110  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...     LibPlate-B6_TS12-A   
111  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...     LibPlate-D6_TS12-B   
112  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...     LibPlate-F6_TS14-A   
113  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...     LibPlate-H6_TS14-B   

     DNA_concentration sample_type fecal_sample     pairs       bases  \
0                 51.5       fecal      AK_SR_1  24276525  6938547858   
1                 28.7       fecal      AK_SR_2  25189547  7192762783   
3                 92.8       fecal      AK_SR_2  11357247  3183231459   
5                411.8       fecal      AK_SR_3   5682989  1595248519   
6                 83.3       fecal      AK_SR_4   5552061  1578724021   
..                 ...         ...          ...       ...         ...   
109                NaN  enrichment      AK_SW_8   6058407  1715849529   
110                NaN  enrichment     AK_SW_12  10370419  2951418443   
111                NaN  enrichment     AK_SW_12   4568279  1283287093   
112                NaN  enrichment     AK_SW_14   2789460   786041399   
113                NaN  enrichment     AK_SW_14   6605728  1864917112   

     raw_pairs    raw_bases          RL  sample_name individual_id sample_id  \
0     40041788  11692202096  142.906529          SR1           SR1   st.t.R1   
1     30917233   9027832036  142.772770        SR2-2         SR2.2   st.R2.2   
3     20045323   5853234316  140.140980        SR2-2         SR2.2   st.R2.2   
5     11410676   3331917392  140.352948          SR3           SR3     st.R3   
6      6520759   1904061628  142.174593          SR4           SR4     st.R4   
..         ...          ...         ...          ...           ...       ...   
109   10693765   3122579380  141.608968   AK-SW8_S59            W8     st.W8   
110   15786166   4609560472  142.299865  AK-SW12_S62           W12    st.W12   
111    8792416   2567385472  140.456296  AK-SW12_S62           W12    st.W12   
112    3550125   1036636500  140.894904  AK-SW14_S63           W14    st.W14   
113   12453511   3636425212  141.159090  AK-SW14_S63           W14    st.W14   

    Gender   Age    BMI  Tribe  Village      Region  Terrain  \
0        F  31.0    NaN  Kabui  Longsai  North-East    Hills   
1        M  55.0    NaN  Kabui  Longsai  North-East    Hills   
3        M  55.0    NaN  Kabui  Longsai  North-East    Hills   
5        F  35.0    NaN  Kabui  Longsai  North-East    Hills   
6        M  39.0    NaN  Kabui  Longsai  North-E

In [5]:
import io
import boto3

def get_matching_s3_objects(bucket, prefix="", suffix=""):
    """
    Generate objects in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch objects whose key starts with
        this prefix (optional).
    :param suffix: Only fetch objects whose keys end with
        this suffix (optional).
    """
    s3 = boto3.client("s3")
    paginator = s3.get_paginator("list_objects_v2")

    kwargs = {'Bucket': bucket}

    # We can pass the prefix directly to the S3 API.  If the user has passed
    # a tuple or list of prefixes, we go through them one by one.
    if isinstance(prefix, str):
        prefixes = (prefix, )
    else:
        prefixes = prefix

    for key_prefix in prefixes:
        kwargs["Prefix"] = key_prefix

        for page in paginator.paginate(**kwargs):
            try:
                contents = page["Contents"]
            except KeyError:
                return

            for obj in contents:
                key = obj["Key"]
                if key.endswith(suffix):
                    yield obj


def get_matching_s3_keys(bucket, prefix="", suffix=""):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    :param suffix: Only fetch keys that end with this suffix (optional).
    """
    for obj in get_matching_s3_objects(bucket, prefix, suffix):
        yield obj["Key"]
        
def check_s3_file(floc):
    '''
    Return True if exists and False if it does not
    '''
    bucket = floc.split('/')[2]
    prefix = '/'.join(floc.split('/')[3:])
    
    found = False
    for key in get_matching_s3_keys(bucket, prefix):
        if prefix in key:
            found = True
    return found

def store_s3_file(bucket, location, binary_string):
    s3 = boto3.resource('s3')
    object = s3.Object(bucket, location)
    object.put(Body=binary_string)
    
def load_coverage_report(s3_bucket, s3_key, sep='\t', names=None):
    '''
    https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
    '''
    # Load the data from s3
    client = boto3.client("s3")
    obj = client.get_object(Bucket=s3_bucket, Key=s3_key)
    df = pd.read_csv(io.BytesIO(obj['Body'].read()), sep=sep, names=names)

    return df

def load_coverage_report2(s3_loc, sep='\t', names=None):
    '''
    https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
    '''
    s3_bucket = s3_loc.split('/')[2]
    s3_key = '/'.join(s3_loc.split('/')[3:])
    
    # Load the data from s3
    client = boto3.client("s3")
    obj = client.get_object(Bucket=s3_bucket, Key=s3_key)
    df = pd.read_csv(io.BytesIO(obj['Body'].read()), sep=sep, names=names)

    return df
    
def read_s3_file(s3_bucket, s3_key):
    s3 = boto3.resource('s3')
    obj = s3.Object(s3_bucket, s3_key)
    return obj.get()['Body'].read().decode("utf-8")

def object_size(s3_bucket, s3_key):    
    return boto3.resource('s3').Bucket(s3_bucket).Object(s3_key).content_length

def object_size2(s3_loc):
    s3_bucket = s3_loc.split('/')[2]
    s3_key = '/'.join(s3_loc.split('/')[3:])
    
    return boto3.resource('s3').Bucket(s3_bucket).Object(s3_key).content_length
    
def read_s3_file2(s3_loc):
    s3 = boto3.resource('s3')
    bucket = s3_loc.split('/')[2]
    key = '/'.join(s3_loc.split('/')[3:])
    return read_s3_file(bucket, key)

def store_s3_file2(s3_loc, binary_string):
    s3 = boto3.resource('s3')
    bucket = s3_loc.split('/')[2]
    key = '/'.join(s3_loc.split('/')[3:])
    return store_s3_file(bucket, key, binary_string)

In [6]:

import io
import boto3

def get_matching_s3_objects(bucket, prefix="", suffix=""):
    """
    Generate objects in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch objects whose key starts with
        this prefix (optional).
    :param suffix: Only fetch objects whose keys end with
        this suffix (optional).
    """
    s3 = boto3.client("s3")
    paginator = s3.get_paginator("list_objects_v2")

    kwargs = {'Bucket': bucket}

    # We can pass the prefix directly to the S3 API.  If the user has passed
    # a tuple or list of prefixes, we go through them one by one.
    if isinstance(prefix, str):
        prefixes = (prefix, )
    else:
        prefixes = prefix

    for key_prefix in prefixes:
        kwargs["Prefix"] = key_prefix

        for page in paginator.paginate(**kwargs):
            try:
                contents = page["Contents"]
            except KeyError:
                return

            for obj in contents:
                key = obj["Key"]
                if key.endswith(suffix):
                    yield obj


def get_matching_s3_keys(bucket, prefix="", suffix=""):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    :param suffix: Only fetch keys that end with this suffix (optional).
    """
    for obj in get_matching_s3_objects(bucket, prefix, suffix):
        yield obj["Key"]
        
def check_s3_file(floc):
    '''
    Return True if exists and False if it does not
    '''
    bucket = floc.split('/')[2]
    prefix = '/'.join(floc.split('/')[3:])
    
    found = False
    for key in get_matching_s3_keys(bucket, prefix):
        if prefix in key:
            found = True
    return found

def store_s3_file(bucket, location, binary_string):
    s3 = boto3.resource('s3')
    object = s3.Object(bucket, location)
    object.put(Body=binary_string)
    
def load_coverage_report(s3_bucket, s3_key, sep='\t', names=None):
    '''
    https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
    '''
    # Load the data from s3
    client = boto3.client("s3")
    obj = client.get_object(Bucket=s3_bucket, Key=s3_key)
    df = pd.read_csv(io.BytesIO(obj['Body'].read()), sep=sep, names=names)

    return df

def load_coverage_report2(s3_loc, sep='\t', names=None):
    '''
    https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
    '''
    s3_bucket = s3_loc.split('/')[2]
    s3_key = '/'.join(s3_loc.split('/')[3:])
    
    # Load the data from s3
    client = boto3.client("s3")
    obj = client.get_object(Bucket=s3_bucket, Key=s3_key)
    df = pd.read_csv(io.BytesIO(obj['Body'].read()), sep=sep, names=names)

    return df
    
def read_s3_file(s3_bucket, s3_key):
    s3 = boto3.resource('s3')
    obj = s3.Object(s3_bucket, s3_key)
    return obj.get()['Body'].read().decode("utf-8")

def object_size(s3_bucket, s3_key):    
    return boto3.resource('s3').Bucket(s3_bucket).Object(s3_key).content_length

def object_size2(s3_loc):
    s3_bucket = s3_loc.split('/')[2]
    s3_key = '/'.join(s3_loc.split('/')[3:])
    
    return boto3.resource('s3').Bucket(s3_bucket).Object(s3_key).content_length
    
def read_s3_file2(s3_loc):
    s3 = boto3.resource('s3')
    bucket = s3_loc.split('/')[2]
    key = '/'.join(s3_loc.split('/')[3:])
    return read_s3_file(bucket, key)

def store_s3_file2(s3_loc, binary_string):
    s3 = boto3.resource('s3')
    bucket = s3_loc.split('/')[2]
    key = '/'.join(s3_loc.split('/')[3:])
    return store_s3_file(bucket, key, binary_string)

def submit_aegea_job(cmd, expected_output, rdb=None, alocation=None, verbose=False):
    """
    Submit the cmd to aegea, capture the jobID, and store the jobID at ~/aegea_logs.txt
    
    v1.0 - 1/26/21
    
    aegea 2.6.9
    """
    from datetime import datetime
    import subprocess
    
    if alocation is None:
        alocation = '/home/mattolm/.aegea_logs.txt'
    
    #1) Check it output already exists
    if check_s3_file(result):
        if verbose:
            print(f"{result} already exists")
        return None
    
    #2) Check if job is already running
    
    # Get list of running jobs
    if rdb is None:
        rdb = load_running_aegea(verbose=False)
    running_jobs = set(rdb[rdb['job_status'].isin(['RUNNING', 'RUNNABLE', 'STARTING'])]['job_ID'].tolist())
    if verbose:
        print(f"{len(running_jobs)} jobs are running")
    
    
    # Get output -> job key
    adb = pd.read_csv(alocation, sep='\t', names=['job_ID', 'output', 'time', 'cmd'])
    cdb = adb[adb['output'] == expected_output]
    if len(cdb) > 0:
        db = cdb[cdb['job_ID'].isin(running_jobs)]
    else:
        db = pd.DataFrame()
    if verbose:
        print(f"Captured {len(adb)} aegea logs, {len(running_jobs)} running jobs, {len(cdb)} previous attempts, {len(db)} currently running attempts")
    
    if len(db) > 0:
        if verbose:
            print(f"{expected_output} is currently running (job={db['job_ID'].tolist()})")
        return None
    
    # 3) Run job
    out = subprocess.check_output(cmd, shell=True, text=True)
    ID = eval(out)['jobId']
    
    # 4) Store job ID
    now = datetime.now()
    dt_string = now.strftime("%d.%m.%Y %H:%M:%S")
    with open(alocation, 'a') as o:
        o.write(f"\n{ID}\t{expected_output}\t{dt_string}\t{cmd}")
        
    # 5) Finish
    if verbose:
        print(f"Job {ID} is launched to create {expected_output}")
    return ID

def load_running_aegea(queue="novome_spot", tries=20, verbose=True):
    """
    Return a list of running aegea jobs
    
    v1.2 - 10/11/21
    * If no queue is specified, dont do a queue
    
    v1.1 - 3/15/21
    * Better printing of status while running showq
    
    v1.0 - 1/26/21
    
    aegea 2.6.9
    """
    import subprocess
    print("Running showq...")
    if queue != "":
        cmd = f"aegea batch ls --queue {queue}"
    else:
         cmd = f"aegea batch ls"
    while tries >= 0:
        try:
            
            raw_out = subprocess.check_output(cmd, shell=True, text=True)
            break
        except:
            #print(f"showq failed for {queue}, try # {tries}")
            tries = tries - 1
    print("Showq succeeded")

    table = {'job_ID':[], 'job_status':[], 'image':[]}
    lines = len(raw_out.split('\n'))
    for j, line in enumerate(raw_out.split('\n')):
        # Skip header
        if j >= 3:
            
            lw = line.strip().split('│')
            
            # Skip weird lines
            if len(lw) != 14:
                continue
            
            table['job_ID'].append(lw[1].strip())
            table['job_status'].append(lw[4].strip())
            table['image'].append(lw[8].strip())

    db = pd.DataFrame(table)
    
    if verbose:
        print(f"{len(db[db['job_status'] == 'RUNNING'])} aegea jobs are currently running")
    
    return db

def make_drep_command(results_dir, **kwargs):
    '''
    Generate an aegea dRep command
    
   https://github.com/SonnenburgLab/docker_images/tree/master/drep_docker
    
    Version 1.1 - 3.1.2021
    - add genome folder option
    
    Version 1.0 - 2.24.2021
    
    Example command:
    
    CMD = ./prepare.sh; conda activate drep; pip install drep --upgrade; ./run_drep.py --drep_command compare --genome_list {0} --results_directory {1} --cmd_args='--S_algorithm fastANI'
  
    '''
    # Get s3 arguments
    bucket_id = kwargs.get('bucket_id', "czbiohub-microbiome")
    image = kwargs.get('image', 'sonnenburglab/drep:latest')
    queue = kwargs.get('queue', 'sonnenburg__spot100')
    ret_result = kwargs.get('ret_result', False)
    wrap_cmd = kwargs.get('wrap_cmd', True)
    
    # Get aegea arguments
    ram = kwargs.get('ram', 32000)
    cores = kwargs.get('cores', 16)
    
    # Get cmd aruments
    unzip = kwargs.get('unzip', False)
    genome_list = kwargs.get('genome_list', None)
    genome_folder = kwargs.get('genome_folder', None)
    timeout = kwargs.get('timeout', 86400) # This is 2 days worth of seconds; make None to ignore
    cmd_args = kwargs.get('cmd_args', "--S_algorithm fastANI")
    drep_command = kwargs.get('drep_command', 'compare')
    genome_list_base = kwargs.get('genome_list_loc', 's3://czbiohub-microbiome/Sonnenburg_Lab/mattolm/tmp/')
    
    if '-p' not in cmd_args:
        cmd_args += f' -p {cores}'
    
    # Handle genome list
    if genome_list is None:
        assert genome_folder is not None
        
        if type(genome_folder) == type('hi'):
            genome_folder = [genome_folder]
        else:
            assert type(genome_folder) == type([])
        
        cmd_substr = f' --genomes_folder {" ".join(genome_folder)} '
        
    elif type(genome_list) == type('hi'):
        genome_list_loc = genome_list
        
        cmd_substr = f' --genome_list {genome_list_loc} '
        
        
    elif type(genome_list) == type([]):
        import uuid
        genome_list_loc = genome_list_base + str(uuid.uuid4())
        
        print(f"Creating genome list on aws at {genome_list_loc}")
        store_s3_file2(genome_list_loc, ('\n'.join(genome_list) + '\n').encode('ascii'))
        
        cmd_substr = f' --genome_list {genome_list_loc} '
    
    if unzip:
        cmd_substr += ' --unzip '

    # Generate the base command
    cmd = f"""
        ./prepare.sh; conda activate drep; pip install drep --upgrade; ./run_drep.py 
        --drep_command {drep_command}
        {cmd_substr}
        --results_directory {results_dir}
        --cmd_args='{cmd_args}'
        """.replace('\n', ' ')
        
    # Remove variable length whitespace
    cmd = ' '.join(cmd.split()) 
    
    # Wrap in aegea
    if timeout is None:
        t = ""
    else:
        t = f"--timeout {timeout}s"
    if wrap_cmd:
        cmd = f"aegea batch submit --queue {queue} --image {image} --storage /mnt=500 --vcpus {cores} --memory {ram} {t} --command=\"{cmd}\" &>> AEGEA.log"
        
    result = os.path.join(results_dir, 'data_tables/Cdb.csv')
    
    if ret_result:
        return cmd, result
        
    else:
        return cmd

## Make dRep command

## Make list of genomes

In [25]:
s2glist = {}

s3_bucket = 'czbiohub-microbiome'

sub_genomes = []
for i, row in Pdb.iterrows():
    

    
    # Get the locations of all genomes
    b_loc = f"Sonnenburg_Lab/IndianMicrobiome/{row['sample']}/08_BINNING/METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGED_K77_METASPADES__{row['sample'].split('.')[0]}/"
    bins = [b for b in get_matching_s3_keys(s3_bucket, b_loc)]
    if len(bins) == 0:
        print(f"{row['sample']} has no bins")
        continue
    assert len(bins) >= 3, b_loc
    
    
    loosers = ['lowDepth', 'tooShort', 'unbinned']
    #print("{0} has {1} genomes".format(row['sample'], len(bins) - 3))
    
    for b in bins:
        keep = True
        for l in loosers:
            if l in b:
                keep = False
        if not keep:
            continue
        assert 'lowDepth' not in b
        sub_genomes.append('s3://' + s3_bucket + '/' + b)
        
print("There are {0} genomes total".format(len(sub_genomes)))

# Write the location to a file
list_loc = "Sonnenburg_Lab/IndianMicrobiome/XDATA_ANALYSIS/DEREPLICATION/ALPHA_GENOME_LIST.txt"
store_s3_file(s3_bucket, list_loc, bytes('\n'.join(sub_genomes), encoding='utf-8'))
print('s3://' + s3_bucket + '/' + list_loc)
    

AK_SK_24 has no bins
AK_SK_34 has no bins
There are 1672 genomes total
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/XDATA_ANALYSIS/DEREPLICATION/ALPHA_GENOME_LIST.txt


## Launch dRep job

In [29]:
ll = 's3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/XDATA_ANALYSIS/DEREPLICATION/ALPHA_GENOME_LIST.txt'
out_loc = 's3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/XDATA_ANALYSIS/DEREPLICATION/genomeSetAlpha/AlphadRep_v1'

rdb = load_running_aegea(queue='sonnenburg__onDemand')
cmd, result = make_drep_command(out_loc, ret_result=True, cmd_args="-sa 0.95 --S_algorithm fastANI -comp 50 -con 15 -d ",
                               ram=128000, rdb=rdb, drep_command='dereplicate', genome_list=ll, queue='sonnenburg__onDemand', unzip=True)

submit_aegea_job(cmd, result, verbose=True, rdb=rdb)


Running showq...
Showq succeeded
0 aegea jobs are currently running


/home/mattolm/miniconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


0 jobs are running
Captured 23427 aegea logs, 0 running jobs, 0 previous attempts, 0 currently running attempts
Job ab1b2481-99ad-419d-85b4-66fddb6021aa is launched to create s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/XDATA_ANALYSIS/DEREPLICATION/genomeSetAlpha/AlphadRep_v1/data_tables/Cdb.csv


'ab1b2481-99ad-419d-85b4-66fddb6021aa'

## Load results

In [33]:
Cdb = load_coverage_report2('s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/XDATA_ANALYSIS/DEREPLICATION/genomeSetAlpha/AlphadRep_v1/drep_output/data_tables/Cdb.csv', sep=',')
Cdb


genome secondary_cluster  \
0    METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGE...               1_1   
1    METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGE...               1_1   
2    METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGE...               1_1   
3    METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGE...               1_1   
4    METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGE...               2_1   
..                                                 ...               ...   
755  METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGE...             205_1   
756  METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGE...             205_2   
757  METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGE...             206_0   
758  METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGE...             207_0   
759  METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGE...             208_0   

     threshold cluster_method comparison_algorithm  primary_cluster  
0         0.05        average              fastANI                1  
1         0.05        average              fastANI                1  
2         0.05        average              fastANI                1  
3         0.05        average              fastANI                1  
4         0.05        average              fastANI                2  
..         ...            ...                  ...              ...  
755       0.05        average              fastANI              205  
756       0.05        average              fastANI              205  
757       0.05        average              fastANI              206  
758       0.05        average              fastANI              207  
759       0.05        average              fastANI              208  

[760 rows x 6 columns]

## Run GTDB

In [7]:
def make_GTDB_command(genome_loc, out_loc, **kwargs):
    """
    aegea batch submit --queue microbiome-lhighPriority --image sonnenburglab/gtdb:latest --storage /mnt=500 --memory 128000 --vcpus 16 --command="export genomes=s3://czbiohub-microbiome/Sonnenburg_Lab/bmerrill/200527_GTDBTK_Analysis/test/; export s3OutputPath=s3://czbiohub-microbiome/Sonnenburg_Lab/bmerrill/200527_GTDBTK_Analysis/test_results/; export coreNum=16; export binExtension=fa; run_gtdb.sh
    """
    # Get s3 arguments
    bucket_id = kwargs.get('bucket_id', "czbiohub-microbiome")
    image = kwargs.get('image', 'sonnenburglab/gtdb:latest')
    queue = kwargs.get('queue', 'sonnenburg__spot100')
    ret_result = kwargs.get('ret_result', False)
    wrap_cmd = kwargs.get('wrap_cmd', True)
    
    # Get aegea arguments
    ram = kwargs.get('ram', 178000)
    cores = kwargs.get('cores', 16)
    timeout = kwargs.get('timeout', 86400) # This is 2 days worth of seconds; make None to ignore
    
    # Get cmd aruments

    # Generate the base command
    cmd = f"""
        export genomes={genome_loc};
        export s3OutputPath={out_loc};
        export coreNum={cores};
        export binExtension=fa;
        ./run_gtdb.sh
        """.replace('\n', ' ')
        
    # Remove variable length whitespace
    cmd = ' '.join(cmd.split()) 
    
    # Wrap in aegea
    if timeout is None:
        t = ""
    else:
        t = f"--timeout {timeout}s"
    if wrap_cmd:
        cmd = f"aegea batch submit --queue {queue} --image {image} --storage /mnt=500 --vcpus {cores} --memory {ram} {t} --command=\"{cmd}\" &>> AEGEA.log"
        
    result = os.path.join(out_loc, 'gtdbtk.bac120.summary.tsv')
    
    if ret_result:
        return cmd, result
        
    else:
        return cmd

In [8]:
out_loc = 's3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/XDATA_ANALYSIS/DEREPLICATION/genomeSetAlpha/annotation/GTDB/'
genome_loc = 's3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/XDATA_ANALYSIS/DEREPLICATION/genomeSetAlpha/AlphadRep_v1/drep_output/dereplicated_genomes/'

cmd, result = make_GTDB_command(genome_loc, out_loc, ret_result=True, verbose=True, ram=240000)

submit_aegea_job(cmd, result)


Running showq...
Showq succeeded


'51535f6d-0eda-4344-a97e-1fc953615326'

## Load

In [17]:
pd.set_option('display.max_colwidth', None)

In [19]:
Rdb = load_coverage_report2(result)
del Rdb['other_related_references(genome_id,species_name,radius,ANI,AF)']

In [24]:
Rdb['species'] = [x.split(';')[-1][3:] for x in Rdb['classification']]
Rdb['species'].value_counts()


                           18
Dialister sp000434475       7
SFJ001 sp004555865          2
Enterococcus_B lactis       2
RC9 sp000432515             2
                           ..
CAG-83 sp900548615          1
UBA1436 sp900540405         1
CAG-83 sp000435555          1
Ruminococcus_E bromii_B     1
Sutterella seckii_A         1
Name: species, Length: 210, dtype: int64

## Run dRep with UHGG (just compare)

In [22]:
FF_loc = 's3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/XDATA_ANALYSIS/DEREPLICATION/genomeSetAlpha/AlphadRep_v1/drep_output/dereplicated_genomes/'

# UHGG genomes
UGHH_loc = 's3://czbiohub-microbiome/Sonnenburg_Lab/Databases/MO_UHGG_v1/individual_genomes/'

In [23]:
out_loc = 's3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/XDATA_ANALYSIS/DEREPLICATION/genomeSetAlpha/AlphadRep_v2'

rdb = load_running_aegea(queue='sonnenburg__spot100')
cmd, result = make_drep_command(out_loc, ret_result=True, cmd_args="-sa 0.95 --multiround_primary_clustering --S_algorithm fastANI",
                               ram=200000, rdb=rdb, drep_command='compare', genome_folder=[FF_loc, UGHH_loc] , queue='sonnenburg__spot100', unzip=False)

submit_aegea_job(cmd, result, verbose=True, rdb=rdb)


Running showq...
Showq succeeded
0 aegea jobs are currently running
0 jobs are running
Captured 23432 aegea logs, 0 running jobs, 0 previous attempts, 0 currently running attempts
Job be7ff850-f72c-415d-9d89-a7e439bdbc97 is launched to create s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/XDATA_ANALYSIS/DEREPLICATION/genomeSetAlpha/AlphadRep_v2/data_tables/Cdb.csv


'be7ff850-f72c-415d-9d89-a7e439bdbc97'

# Make table of genomes

                           18
Dialister sp000434475       7
SFJ001 sp004555865          2
Enterococcus_B lactis       2
RC9 sp000432515             2
                           ..
CAG-83 sp900548615          1
UBA1436 sp900540405         1
CAG-83 sp000435555          1
Ruminococcus_E bromii_B     1
Sutterella seckii_A         1
Name: species, Length: 210, dtype: int64

In [21]:
Rdb[Rdb['species'] == '']

user_genome  \
0     METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGED_K77_METASPADES__AK_SG_10.3   
21    METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGED_K77_METASPADES__AK_SG_2.20   
48   METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGED_K77_METASPADES__AK_SK_10.20   
56    METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGED_K77_METASPADES__AK_SK_13.5   
59   METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGED_K77_METASPADES__AK_SK_15.17   
67    METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGED_K77_METASPADES__AK_SK_25.3   
70    METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGED_K77_METASPADES__AK_SK_26.3   
79    METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGED_K77_METASPADES__AK_SK_33.8   
97   METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGED_K77_METASPADES__AK_SK_40.15   
120   METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGED_K77_METASPADES__AK_SK_5.27   
125   METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGED_K77_METASPADES__AK_SK_5.38   
151    METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGED_K77_METASPADES__AK_SR_4.1   
180   METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGED_K77_METASPADES__AK_SW_5.20   
182   METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGED_K77_METASPADES__AK_SW_5.25   
195    METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGED_K77_METASPADES__RS12_A.12   
215      METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGED_K77_METASPADES__RS5_A.4   
227     METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGED_K77_METASPADES__TS14_B.2   
233     METABAT215_SUBJECTMAPPING_SCAFFOLDS_1500_MERGED_K77_METASPADES__TS5_A.10   

                                                                                                    classification  \
0    d__Bacteria;p__Actinobacteriota;c__Coriobacteriia;o__Coriobacteriales;f__Coriobacteriaceae;g__Collinsella;s__   
21                          d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__CAG-272;g__UBA1740;s__   
48          d__Bacteria;p__Firmicutes_C;c__Negativicutes;o__Veillonellales;f__Veillonellaceae;g__Veillonella_A;s__   
56                 d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Prevotella;s__   
59   d__Bacteria;p__Actinobacteriota;c__Coriobacteriia;o__Coriobacteriales;f__Coriobacteriaceae;g__Collinsella;s__   
67             d__Bacteria;p__Firmicutes_A;c__Clostridia_A;o__Christensenellales;f__Borkfalkiaceae;g__UBA10281;s__   
70   d__Bacteria;p__Cyanobacteria;c__Vampirovibrionia;o__Gastranaerophilales;f__Gastranaerophilaceae;g__QHMH01;s__   
79   d__Bacteria;p__Actinobacteriota;c__Coriobacteriia;o__Coriobacteriales;f__Coriobacteriaceae;g__Collinsella;s__   
97             d__Bacteria;p__Firmicutes_A;c__Clostridia_A;o__Christensenellales;f__Borkfalkiaceae;g__UBA10281;s__   
120  d__Bacteria;p__Actinobacteriota;c__Coriobacteriia;o__Coriobacteriales;f__Coriobacteriaceae;g__Collinsella;s__   
125    d__Bacteria;p__Patescibacteria;c__Saccharimonadia;o__Saccharimonadales;f__Saccharimonadaceae;g__UBA1103;s__   
151  d__Bacteria;p__Cyanobacteria;c__Vampirovibrionia;o__Gastranaerophilales;f__Gastranaerophilaceae;g__Zag111;s__   
180            d__Bacteria;p__Firmicutes_A;c__Clostridia_A;o__Christensenellales;f__Borkfalkiaceae;g__UBA10281;s__   
182                d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__CAG-110;s__   
195  d__Bacteria;p__Actinobacteriota;c__Coriobacteriia;o__Coriobacteriales;f__Coriobacteriaceae;g__Collinsella;s__   
215           d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Ruminococcaceae;g__Pygmaiobacter;s__   
227               d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Bacteroides;s__   
233               d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__UMGS1202;s__   

    fastani_reference  fastani_reference_radius fastani_taxonomy  fastani_ani  \
0                 NaN                       NaN              NaN          NaN   
21                NaN                       NaN              NaN   